<a href="https://colab.research.google.com/github/Viktory4121/MORD/blob/main/%D0%94%D0%9E%D0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Динамические окрестностные модели (ДОМ)
####Вариант 5

In [72]:
import numpy as np
import pandas as pd
import random
import copy
import math
from prettytable import PrettyTable
import itertools as it

In [73]:
#A={a1, a2, a3}

#какие дуги входят в позицию:
#Ox[a1]={a1}
#Ox[a2]={}
#Ox[a3]={a1, a2}

#в какие позиции входят v
#Ov[a1]={a1}
#Ov[a2]={a2}
#Ov[a3]={a3}

#количества состояний/входов/выходов
elem = 100
count = [3, 3, 3]
Sx = np.array([[1, 0, 1], [0, 0, 1], [0, 0, 0]])
Sv = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

print("Матрица смежности:")
print("Sx = ")
print(Sx)
print("Sv = ")
print(Sv)

Матрица смежности:
Sx = 
[[1 0 1]
 [0 0 1]
 [0 0 0]]
Sv = 
[[1 0 0]
 [0 1 0]
 [0 0 1]]


In [74]:
def x1(x, v):
  return math.sin(math.log(abs(v[1] + v[0])) / x[2])

def x2(x, v):
  return math.cos(v[0] * x[2] - (x[1] / x[2])**3)

def x3(x, v):
  return math.sin(math.tan(v[0] / v[2]) - 3**(x[2]))

def v1(x, v):
  return (x[1] * v[0] * v[1] + math.tan(x[2])**2)

def v2(x, v):
  return (0.2 * v[0] + 33.0 * v[1] + v[2])

def v3(x, v):
  return (math.exp(x[0]) + .45 * math.log2(abs(x[1] * x[2])))

def y1(x, v):
  return (x[0] / v[0]) * math.sqrt(abs(x[0] / x[1]))

def y2(x, v):
  return math.sqrt(abs(math.sin(v[2])))

def y3(x, v):
  return v[2] * v[0] / v[0]

functions = [x1, x2, x3, v1, v2, v3, y1, y2, y3]

In [75]:
#Метод псевдообращения Бена-Израиля
def BenIsrael(A, eps = 0.000001): #А - np.array, eps - точность вычислительная
  I = np.diag(np.diag(np.ones([len(A),len(A)])))
  buf = 1
  i = 1

  if(np.linalg.norm(A, ord='fro') != 0): #если не нулевая норма
    gamma = 1.6 / (np.linalg.norm(A, ord='fro')**2)
    A1 = gamma * (A.T)
    i += 1
    A2 = A1.dot(2*I - A.dot(A1))
    E = np.linalg.norm(A2 - A1, ord=np.inf) / np.linalg.norm(A1, ord=np.inf)
  else: #если нулевая норма
    A1 = np.zeros((len(A[0]), len(A)))
    buf = 0

  while(E >= eps and buf):
    A1 = A2.copy()
    A2 = A1.dot(2*I - A.dot(A1))
    E = np.linalg.norm(A2 - A1, ord=np.inf) / np.linalg.norm(A1, ord=np.inf)
    i += 1

  return A1

#решение СЛАУ вида AX=B
def solvingSLAE(A, B):
  A = np.array(A.copy())

  #проверка на бесконечное количество решений
  A_psev = BenIsrael(A)

  X = A_psev.dot(B)
  return X

#=================================================================
#Генерация случайного вектора из 3-х элементов в диапазоне от -3 до 3
def generationRandomVector(elem = 3, start = -3, end = 3):
  vector = []
  for i in range(elem):
    vector.append(random.uniform(start, end))
  return vector

#=================================================================
#создание столбца таблицы
def createVariants(function, x, v, typeFunc = 1, elem = 100):
  vector = []

  if(typeFunc == 0):
    for i in range(elem):
      vector.append(function(generationRandomVector(), generationRandomVector()))
  else:
    for i in range(elem):
      vector.append(function(x[i], v[i]))

  return vector

#=================================================================
#Создание ДОМ в определённый момент времени
def createTableDNM(functions, count, step, oldTable = []): #count(x, v, y)
  table = []
  vector = []
  columns = createColumns(step, count)

  if(step == 0):
    for i in range(sum(count[0:2])):
      table.append(createVariants(functions[i], [], [], 0)) #xi, vi

  else:
    for i in range(count[0]): #xi
      table.append(createVariants(functions[i], (np.array(oldTable))[:, 0:count[0]], (np.array(oldTable))[:, count[0]:sum(count[0:2])]))
    for i in range(count[1]): #vi
      table.append(createVariants(functions[count[0] + i], [], [], 0))

  for i in range(count[2]): #yi
    table.append(createVariants(functions[sum(count[0:2]) + i], (np.array(table).T)[:, 0:count[0]], (np.array(table).T)[:, count[0]:sum(count[0:2])]))

  table = pd.DataFrame(np.array(table).T, columns = columns)
  return table

#заполнение
def createColumns(step, count):
  columns = []
  for i in range(count[0]):
    columns.append("x" + str(i + 1) + "(" + str(step) + ")")
  for i in range(count[1]):
    columns.append("v" + str(i + 1) + "(" + str(step) + ")")
  for i in range(count[2]):
    columns.append("y" + str(i + 1) + "(" + str(step) + ")")
  return columns

In [76]:
#functions = [x1, x2, x3, v1, v2, v3, y1, y2, y3]
model_0 = createTableDNM(functions, count, 0)
model_1 = createTableDNM(functions, count, 1, np.array(model_0))
model_2 = createTableDNM(functions, count, 2, np.array(model_1))
model_3 = createTableDNM(functions, count, 3, np.array(model_2))
model_4 = createTableDNM(functions, count, 4, np.array(model_3))
model_5 = createTableDNM(functions, count, 5, np.array(model_4))
model_6 = createTableDNM(functions, count, 6, np.array(model_5))
model_7 = createTableDNM(functions, count, 7, np.array(model_6))
model_8 = createTableDNM(functions, count, 8, np.array(model_7))
model_9 = createTableDNM(functions, count, 9, np.array(model_8))
model_10 = createTableDNM(functions, count, 10, np.array(model_9))

In [77]:
model_9

,x1(9),x2(9),x3(9),v1(9),v2(9),v3(9),y1(9),y2(9),y3(9)
0,0.469845,-0.942072,0.963419,2.643854,-49.475967,11.771485,0.125503,0.844857,11.771485
1,-0.107326,0.965819,-0.131990,0.474265,-60.090941,9.817471,-0.075437,0.618610,9.817471
2,-0.063932,0.988500,-0.504785,270.546936,38.665366,1.383076,-0.000060,0.991177,1.383076
3,-0.700425,0.277967,0.998643,10.773226,79.756835,8.497593,-0.103205,0.894390,8.497593
4,0.676279,-0.770846,-0.511220,0.421513,35.109606,2.865617,1.502773,0.522002,2.865617
...,...,...,...,...,...,...,...,...,...
95,0.721191,0.976302,-0.643860,1.072160,-1.674748,0.460843,0.578127,0.666861,0.460843
96,-0.245027,0.303443,0.752315,0.448561,-91.409025,-1.958736,-0.490865,0.962128,-1.958736
97,-0.292092,-0.014903,-0.560927,134.730360,-75.593481,0.597242,-0.009598,0.749909,0.597242
98,0.208778,0.031806,-0.972119,0.167066,8.429884,0.736359,3.201717,0.819509,0.736359


In [78]:
model_10

,x1(10),x2(10),x3(10),v1(10),v2(10),v3(10),y1(10),y2(10),y3(10)
0,-0.751961,-0.942576,-0.469055,2.296768,-5.877524,6.654942,-0.292427,0.602705,6.654942
1,0.429836,-0.574274,-0.728871,7.497928,-6.783687,1.043817,0.049597,0.929694,1.043817
2,0.934558,-0.968131,0.503143,0.089635,95.025696,4.311351,10.243899,0.959508,4.311351
3,-0.979950,-0.255663,0.201792,2.140336,54.182375,18.885120,-0.896375,0.188565,18.885120
4,-0.644911,-0.876529,-0.409689,3.029104,-42.120446,10.692926,-0.182622,0.977011,10.692926
...,...,...,...,...,...,...,...,...,...
95,0.708023,-0.940903,-0.999861,0.015615,6.767312,12.697290,39.332167,0.361311,12.697290
96,-0.283838,0.963278,-0.583625,2.379618,-51.379578,-1.068917,-0.064747,0.936312,-1.068917
97,-0.836155,0.984616,-0.943965,-9.497332,83.868077,1.413929,0.081133,0.993842,1.413929
98,-0.800712,0.986846,-0.112605,-0.027990,-50.410551,4.813541,25.768620,0.997441,4.813541


##1. Линейная форма функции пересчёта

In [79]:
def buildLinearRegretion(model1, model2, cols):
  vector_ones = [1] * elem
  coef = []

  linearFunction = (model1.copy())[cols[0:(len(cols) - 1)]]
  linearFunction[cols[len(cols) - 1]] = (model2.copy())[cols[len(cols) - 1]]
  linearFunction.insert(0, "optional_col", vector_ones)

  A_linear = np.array(linearFunction)
  #A_psev = BenIsrael(A_linear[:, 0:(len(A_linear[0]) - 1)])
  X_linear = solvingSLAE(A_linear[:, 0:(len(A_linear[0]) - 1)], np.matrix(A_linear[:, len(A_linear[0]) - 1]).T)

  estimate_linear = []
  variance_linear = []
  error_linear = []

  for i in range(len(linearFunction)):
    for j in range(0, len(X_linear)):
      if(j == 0):
        estimate_linear.append(X_linear[j,0])
      else:
        estimate_linear[i] += X_linear[j,0] * np.array(linearFunction)[i,j]

    variance_linear.append((1.0 / len(linearFunction)) * (np.array(linearFunction)[i, len(np.array(linearFunction)[0]) - 1] - estimate_linear[i])**2)
    #error_linear.append((1.0 / len(linearFunction)) * abs((np.array(linearFunction)[i, len(np.array(linearFunction)[0]) - 1] - estimate_linear[i])
                            #/ np.array(linearFunction)[i, len(np.array(linearFunction)[0]) - 1]) * 100)
    error_linear.append((1.0 / len(linearFunction)) * abs((np.array(linearFunction)[i, len(np.array(linearFunction)[0]) - 1] - estimate_linear[i])
                            / max(np.array(linearFunction)[:, len(np.array(linearFunction)[0]) - 1])) * 100)

  linearFunction['regression'] = estimate_linear
  linearFunction['variance'] = variance_linear
  linearFunction['error, %'] = error_linear

  return X_linear, linearFunction

###Для функции $x_1[t+1]$

In [80]:
#x3, v1, v2
X1_linear, X1_linearTable = buildLinearRegretion(model_9, model_10, ['x3(9)', 'v1(9)', 'v2(9)', 'x1(10)'])
print(pd.DataFrame(X1_linear, columns=['X1'], index=['a1', 'c13', 'b11', 'b12']))
X1_linearTable

               X1
a1  -9.574334e-02
c13 -4.783483e-01
b11  4.504683e-08
b12  2.860676e-03


,optional_col,x3(9),v1(9),v2(9),x1(10),regression,variance,"error, %"
0,1,0.963419,2.643854,-49.475967,-0.751961,-0.698128,0.000029,0.054011
1,1,-0.131990,0.474265,-60.090941,0.429836,-0.204507,0.004024,0.636439
2,1,-0.504785,270.546936,38.665366,0.934558,0.256341,0.004600,0.680458
3,1,0.998643,10.773226,79.756835,-0.979950,-0.345284,0.004028,0.636763
4,1,-0.511220,0.421513,35.109606,-0.644911,0.249235,0.007995,0.897100
...,...,...,...,...,...,...,...,...
95,1,-0.643860,1.072160,-1.674748,0.708023,0.207455,0.002506,0.502221
96,1,0.752315,0.448561,-91.409025,-0.283838,-0.717104,0.001877,0.434697
97,1,-0.560927,134.730360,-75.593481,-0.836155,-0.043667,0.006280,0.795105
98,1,-0.972119,0.167066,8.429884,-0.800712,0.393384,0.014259,1.198040


In [81]:
print("E = " + str(sum(X1_linearTable['variance'])))
print("A = " + str(sum(X1_linearTable['error, %'])))

E = 0.39645838003004513
A = 53.329488907349116


###Для функции $x_2[t+1]$

In [82]:
#x2, x3, v1
X2_linear, X2_linearTable = buildLinearRegretion(model_9, model_10, ['x2(9)', 'x3(9)', 'v1(9)', 'x2(10)'])
print(pd.DataFrame(X2_linear, columns=['X2'], index=['a2', 'c22', 'c23', 'b21']))
X2_linearTable

               X2
a2   8.154154e-02
c22  2.215598e-02
c23  1.792870e-01
b21 -3.519917e-08


,optional_col,x2(9),x3(9),v1(9),x2(10),regression,variance,"error, %"
0,1,-0.942072,0.963419,2.643854,-0.942576,0.233397,0.013829,1.176430
1,1,0.965819,-0.131990,0.474265,-0.574274,0.079276,0.004271,0.653804
2,1,0.988500,-0.504785,270.546936,-0.968131,0.012932,0.009625,0.981444
3,1,0.277967,0.998643,10.773226,-0.255663,0.266744,0.002729,0.522610
4,1,-0.770846,-0.511220,0.421513,-0.876529,-0.027192,0.007214,0.849666
...,...,...,...,...,...,...,...,...
95,1,0.976302,-0.643860,1.072160,-0.940903,-0.012263,0.008624,0.929000
96,1,0.303443,0.752315,0.448561,0.963278,0.223145,0.005478,0.740421
97,1,-0.014903,-0.560927,134.730360,0.984616,-0.019360,0.010080,1.004366
98,1,0.031806,-0.972119,0.167066,0.986846,-0.092042,0.011640,1.079308


In [83]:
print("E = " + str(sum(X2_linearTable['variance'])))
print("A = " + str(sum(X2_linearTable['error, %'])))

E = 0.504517045300515
A = 61.91766778655987


###Для функции $x_3[t+1]$

In [84]:
#x3, v1, v3
X3_linear, X3_linearTable = buildLinearRegretion(model_9, model_10, ['x3(9)', 'v1(9)', 'v3(9)', 'x3(10)'])
print(pd.DataFrame(X3_linear, columns=['X3'], index=['a3', 'c33', 'b31', 'b33']))
X3_linearTable

               X3
a3  -2.735583e-01
c33 -1.560012e-01
b31 -2.222589e-08
b33 -9.116351e-03


,optional_col,x3(9),v1(9),v3(9),x3(10),regression,variance,"error, %"
0,1,0.963419,2.643854,11.771485,-0.469055,-0.531166,0.000039,0.062281
1,1,-0.131990,0.474265,9.817471,-0.728871,-0.342467,0.001493,0.387463
2,1,-0.504785,270.546936,1.383076,0.503143,-0.207426,0.005049,0.712515
3,1,0.998643,10.773226,8.497593,0.201792,-0.506815,0.005021,0.710549
4,1,-0.511220,0.421513,2.865617,-0.409689,-0.219931,0.000360,0.190278
...,...,...,...,...,...,...,...,...
95,1,-0.643860,1.072160,0.460843,-0.999861,-0.177317,0.006766,0.824798
96,1,0.752315,0.448561,-1.958736,-0.583625,-0.373064,0.000443,0.211138
97,1,-0.560927,134.730360,0.597242,-0.943965,-0.191501,0.005662,0.754526
98,1,-0.972119,0.167066,0.736359,-0.112605,-0.128619,0.000003,0.016058


In [85]:
print("E = " + str(sum(X3_linearTable['variance'])))
print("A = " + str(sum(X3_linearTable['error, %'])))

E = 0.3838387418793952
A = 52.53450296485351


###Для функции $y_1[t]$

In [86]:
#x1, x2, v1
Y1_linear, Y1_linearTable = buildLinearRegretion(model_9, model_9, ['x1(9)', 'x2(9)', 'v1(9)', 'y1(9)'])
print(pd.DataFrame(Y1_linear, columns=['Y1'], index=['a1', 'c11', 'c12', 'b11']))
Y1_linearTable

               Y1
a1   5.214199e-01
c11 -2.068524e-01
c12  2.587843e-01
b11 -2.063842e-08


,optional_col,x1(9),x2(9),v1(9),y1(9),regression,variance,"error, %"
0,1,0.469845,-0.942072,2.643854,0.125503,0.180438,0.000030,0.003629
1,1,-0.107326,0.965819,0.474265,-0.075437,0.793559,0.007552,0.057411
2,1,-0.063932,0.988500,270.546936,-0.000060,0.790447,0.006249,0.052225
3,1,-0.700425,0.277967,10.773226,-0.103205,0.738238,0.007080,0.055590
4,1,0.676279,-0.770846,0.421513,1.502773,0.182047,0.017443,0.087254
...,...,...,...,...,...,...,...,...
95,1,0.721191,0.976302,1.072160,0.578127,0.624891,0.000022,0.003089
96,1,-0.245027,0.303443,0.448561,-0.490865,0.650631,0.013030,0.075413
97,1,-0.292092,-0.014903,134.730360,-0.009598,0.577980,0.003452,0.038819
98,1,0.208778,0.031806,0.167066,3.201717,0.486465,0.073726,0.179384


In [87]:
print("E = " + str(sum(Y1_linearTable['variance'])))
print("A = " + str(sum(Y1_linearTable['error, %'])))

E = 10.101725703602607
A = 9.02974480080757


###Для функции $y_2[t]$

In [88]:
#v3
Y2_linear, Y2_linearTable = buildLinearRegretion(model_9, model_9, ['v3(9)', 'y2(9)'])
print(pd.DataFrame(Y2_linear, columns=['Y2'], index=['a2', 'b23']))
Y2_linearTable

           Y2
a2   0.701910
b23  0.011496


,optional_col,v3(9),y2(9),regression,variance,"error, %"
0,1,11.771485,0.844857,0.837236,5.808329e-07,0.007622
1,1,9.817471,0.618610,0.814772,3.847966e-04,0.196187
2,1,1.383076,0.991177,0.717810,7.472972e-04,0.273402
3,1,8.497593,0.894390,0.799599,8.985417e-05,0.094804
4,1,2.865617,0.522002,0.734853,4.530567e-04,0.212879
...,...,...,...,...,...,...
95,1,0.460843,0.666861,0.707208,1.627894e-05,0.040352
96,1,-1.958736,0.962128,0.679392,7.993957e-04,0.282772
97,1,0.597242,0.749909,0.708776,1.691952e-05,0.041139
98,1,0.736359,0.819509,0.710375,1.191011e-04,0.109147


In [89]:
print("E = " + str(sum(Y2_linearTable['variance'])))
print("A = " + str(sum(Y2_linearTable['error, %'])))

E = 0.06098039325443533
A = 20.263316685468617


###Для функции $y_3[t]$

In [90]:
#v1, v3
Y3_linear, Y3_linearTable = buildLinearRegretion(model_9, model_9, ['v1(9)', 'v3(9)', 'y3(9)'])
print(pd.DataFrame(Y3_linear, columns=['Y3'], index=['a3', 'b31', 'b33']))
Y3_linearTable

               Y3
a3   2.108530e-08
b31 -8.082347e-16
b33  1.000000e+00


,optional_col,v1(9),v3(9),y3(9),regression,variance,"error, %"
0,1,2.643854,11.771485,11.771485,11.771485,6.317174e-19,4.452312e-10
1,1,0.474265,9.817471,9.817471,9.817471,9.788480e-20,1.752596e-10
2,1,270.546936,1.383076,1.383076,1.383076,3.123647e-18,9.900454e-10
3,1,10.773226,8.497593,8.497593,8.497593,1.605390e-22,7.097647e-12
4,1,0.421513,2.865617,2.865617,2.865617,1.964904e-18,7.852262e-10
...,...,...,...,...,...,...,...
95,1,1.072160,0.460843,0.460843,0.460843,3.979495e-18,1.117476e-09
96,1,0.448561,-1.958736,-1.958736,-1.958736,6.716576e-18,1.451771e-09
97,1,134.730360,0.597242,0.597242,0.597242,3.846363e-18,1.098624e-09
98,1,0.167066,0.736359,0.736359,0.736359,3.712996e-18,1.079410e-09


In [91]:
print("E = " + str(sum(Y3_linearTable['variance'])))
print("A = " + str(sum(Y3_linearTable['error, %'])))

E = 3.0675356164377644e-16
A = 9.180044691481782e-08


##2. Квадратичная форма функции пересчёта